In [2]:
import sys
import os
sys.path.append('..')

In [3]:
from src.dataset import ImageDataLoader
from src import config 
from src.utils import (

    get_test_transform,
    )

import torch
from pathlib import Path
from src.model import Net
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from PIL import Image

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


In [4]:
def load_model_weights(model, model_path):
    checkpoint = torch.load(model_path)
    model.load_state_dict(checkpoint['state_dict'])
    return model

def get_all_image_files(pathlib_root_folder):
    img_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']
    img_regex = re.compile('|'.join(img_extensions), re.IGNORECASE)
    image_files = [f for f in pathlib_root_folder.glob('**/*') if f.is_file() and img_regex.search(f.suffix)]
    return image_files


def predict(model, image):
    model.eval()

    with torch.no_grad():
        
        output = model(image.to(DEVICE))
        scores = torch.sigmoid(output)
        predictions = (scores>0.5).float()
        _, pred = torch.min(predictions, 1)

    return pred.item()

In [5]:
hiper_path = '../artifacts/hiper_others_2023-05-12-18-06-48/4_fold_max_acc_checkpoint.pth.tar'
membran_path = '../artifacts/membran_others_2023-05-12-20-31-18/0_fold_max_acc_checkpoint.pth.tar'
sclero_path = '../artifacts/sclero_others_2023-05-12-13-27-54/4_fold_max_acc_checkpoint.pth.tar'
normal_path = '../artifacts/normal_others_2023-05-12-15-45-47/4_fold_max_acc_checkpoint.pth.tar'



hiper_model = load_model_weights(Net(net_version="b0", num_classes=2).to(DEVICE), hiper_path)
membran_model = load_model_weights(Net(net_version="b0", num_classes=2).to(DEVICE), membran_path)
sclero_model = load_model_weights(Net(net_version="b0", num_classes=2).to(DEVICE), sclero_path)
normal_model = load_model_weights(Net(net_version="b0", num_classes=2).to(DEVICE), normal_path)


transform = get_test_transform()

Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0


In [6]:
result = {
'True Attribute':[],
'Hipercellularity':[],
'Membranous':[],
'Normal':[],
'Sclerosis':[],
'File'    :[],
}
data_path = '../data/raw'
files = get_all_image_files(Path(data_path))

for image_path in files:

    class_name = str(image_path).split('\\')[3]

    image = Image.open(image_path)
    image = transform(image)
    image = image.unsqueeze(0)

    result['True Attribute'].append(class_name)
    result['Hipercellularity'].append(predict(hiper_model, image))
    result['Membranous'].append(predict(membran_model, image))
    result['Normal'].append(predict(normal_model, image))
    result['Sclerosis'].append(predict(sclero_model, image))
    result['File'].append(str(image_path))


In [7]:
df = pd.DataFrame(result)
df

,True Attribute,Hipercellularity,Membranous,Normal,Sclerosis,File
0,hipercellularity,1,0,0,0,..\data\raw\hipercellularity\AZAN\PSHIPERCELUL...
1,hipercellularity,1,0,0,0,..\data\raw\hipercellularity\AZAN\PSHIPERCELUL...
2,hipercellularity,1,0,0,0,..\data\raw\hipercellularity\AZAN\PSHIPERCELUL...
3,hipercellularity,1,0,0,0,..\data\raw\hipercellularity\AZAN\PSHIPERCELUL...
4,hipercellularity,1,0,0,0,..\data\raw\hipercellularity\AZAN\PSHIPERCELUL...
...,...,...,...,...,...,...
10177,sclerosis,0,0,0,1,..\data\raw\sclerosis\sclerosis_withoutMemb\PS...
10178,sclerosis,0,0,0,1,..\data\raw\sclerosis\sclerosis_withoutMemb\PS...
10179,sclerosis,0,0,0,1,..\data\raw\sclerosis\sclerosis_withoutMemb\PS...
10180,sclerosis,0,0,0,1,..\data\raw\sclerosis\sclerosis_withoutMemb\PS...


In [8]:
df.to_csv('../data/raw/data_index_completed.csv',index=False)